[API Documentation](https://financialmodelingprep.com/developer/docs/)
[Python for Finance Video](https://youtu.be/mjyNWNjN6J4)
[Python Waterfall Chart](https://plotly.com/python/waterfall-charts/)

In [1]:
import plotly.graph_objects as go
import pandas as pd
import requests
import json
import os

apikey = os.environ.get('apikey')

In [2]:
def get_income_statement(symbol):
    r = requests.get(f'https://financialmodelingprep.com/api/v3/financials/income-statement/{symbol}?period=quarter&apikey={apikey}')
    r = r.json()
    PnL = r['financials']
    PnL = pd.DataFrame.from_dict(PnL)
    PnL = PnL.T
    PnL.columns = PnL.iloc[0]
    PnL.reset_index(inplace = True)
    PnL = PnL.iloc[:,0:2]
    PnL.rename(columns = {PnL.columns[1]:symbol},inplace = True)
    cols = PnL.columns.drop('index')
    PnL[cols] = PnL[cols].apply(pd.to_numeric, errors='coerce')
    PnL = PnL.iloc[1:,]
    return PnL

In [3]:
PnL=get_income_statement('ZM')

In [4]:
Revenue = PnL[PnL['index'] == 'Revenue'].iloc[0][1]
COGS = PnL[PnL['index'] == 'Cost of Revenue'].iloc[0][1]*-1
GrossProfit = PnL[PnL['index'] == 'Gross Profit'].iloc[0][1] 
RD = PnL[PnL['index'] == 'R&D Expenses'].iloc[0][1]*-1 
SGA = PnL[PnL['index'] == 'SG&A Expense'].iloc[0][1]*-1
OperatingIncome = PnL[PnL['index'] == 'Operating Income'].iloc[0][1]*-1 
Interest = PnL[PnL['index'] == 'Interest Expense'].iloc[0][1]*-1
EBT = PnL[PnL['index'] == 'Earnings before Tax'].iloc[0][1]
IncomeTax = PnL[PnL['index'] == 'Income Tax Expense'].iloc[0][1]*-1
NetIncome = PnL[PnL['index'] == 'Net Income'].iloc[0][1] 

In [7]:
fig = go.Figure(go.Waterfall(
    name = '20', orientation = 'v',
    measure = ['relative','relative','total','relative','relative','total','relative','total','relative','total'],
    x = ['Revenue','COGS','Gross Profit','R&D','SG&A','Operating Income','Interest Expense','Earnings Before Tax','Income Tax','Net Income'],
    textposition = 'outside',
    text = [Revenue/100000,COGS/100000,GrossProfit/100000,RD/100000,SGA/1000000,OperatingIncome/1000000,Interest/100000,EBT/100000,IncomeTax/100000,NetIncome/100000],
    y = [Revenue,COGS,GrossProfit,RD,SGA,OperatingIncome,Interest,EBT,IncomeTax,NetIncome],
    connector = {'line':{'color':'rgb(63, 63, 63)'}},))
fig.update_layout(
    title = 'Profit and Loss Statement',
    showlegend = True)
fig.show()